#### PSF ANALYSIS

Code to extract the sigma values of a 3D Gaussian-modelled PSF, used for 3D deconvolution (Step 0  of data analuysis)

In [ ]:
"""
Code adapted from D.Nguyen 2021:
    https://github.com/Omnistic/MicrobeadsToPSF.git
"""
import numpy as np
import matplotlib.pyplot as plt
import importlib
import pickle
from scipy.optimize import curve_fit
from scipy.optimize import curve_fit
# local folder with homemade libraries
import sys
sys.path.append('/home/ngc/Documents/GitHub/multiview_utilities')
# HOMEMADE LIBRARIES AND IMPORTS
import math_utils as matut
import raw_images_manipulation_utilities as imanip
importlib.reload(matut); # 
importlib.reload(manip); # reload the modules for updating to eventual changes

#### Local variables and constants.

Carefully read and change the variables in the next sectionto match the volume to be analysed.

In [ ]:
VOLUME_SLICES = 41 # number of images for each volume
TIMEPOINTS =  1
TOTAL_IMAGES = TIMEPOINTS * VOLUME_SLICES # total of raw data-images
IMAGES_DIMENSION = 2304 # assumed square images. N.b. try to have 2^n pixels
RAW_SLICES_FOLDER = '/home/ngc/Desktop/test_data/Beads20X_LowerC/beads'
BACKGROUND_FOLDER = '/home/ngc/Desktop/test_data/Beads20X_LowerC/beads'

Define the cube around each bead (this volume will be the fitting dominion).

*Choose sides that are slightly larger than the bead profile, eyeballing it looking thorugh the stack. Does not have to be precise. A pixel larger is better than a pixel smaller. In general, the z component will be the largest one, controlling the extent of the box.*

In [ ]:
box_side = 21
box = np.asarray((box_side, box_side, box_side)

##### How may beads to consider?

(code will stop after finding this number of beads)

In [ ]:
max_beads = 20

#### Variables

In [ ]:
z_width, x_width, y_width = [], [], []

data_list = rim.file_names_list(TIMEPOINTS)

stack_0 = rim.open_binary_stack(
    RAW_SLICES_FOLDER + data_list[0][1],
    BACKGROUND_FOLDER + '/Background_0.tif',
    size_x=IMAGES_DIMENSION,
    size_y=IMAGES_DIMENSION)

stack_1 = rim.open_binary_stack(
    RAW_SLICES_FOLDER + data_list[0][2],
    BACKGROUND_FOLDER + '/Background_1.tif',
    size_x=IMAGES_DIMENSION,
    size_y=IMAGES_DIMENSION)

In [ ]:
x_val = np.arange(0, box[0], 1)
y_val = np.arange(0, box[1], 1)
z_val = np.arange(0, box[2], 1)
z_grid, x_grid, y_grid = np.meshgrid(z_val, x_val, y_val, indexing='ij')
xdata = np.vstack((z_grid.ravel(), x_grid.ravel(), y_grid.ravel()))
mean_b_z, mean_b_x, mean_b_y = 0, 0, 0

#### Beads seach and Gaussian fitting

(lopping in the **while**)

In [ ]:
beads = 0
z_chosen = int(stack_0.shape[0] / 2)
print('\nNUMBER OF BEADS FOUND:\n')

while beads < max_beads:

    x_max, y_max = np.unravel_index(
        np.argmax(stack_1[20, :, :]),
        stack_1[20, :, :].shape)

    if (
        x_max - box[1] / 2 > 0
        and x_max + box[1] / 2 < stack_0.shape[1]
        and y_max - box[2] / 2 > 0
        and y_max + box[2] / 2 < stack_0.shape[2]
        ):
        substack = stack_1[
            int(z_chosen - box[0] / 2):int(z_chosen + box[0] / 2),
            int(x_max - box[1] / 2):int(x_max + box[1] / 2),
            int(y_max - box[2] / 2):int(y_max + box[2] / 2),
            ]
        # Reasonable starting parameters, assuming that the PSF is sampled by a few pixels (3) in x-y
        # and 5 in z. Again these parameters are not crucial, they just need to be in a close-ish range
        # of the actual data, in order for the fitting to run smoothly
        
        bg = 1 # background
        A_coeff = 1000 # bead intensity peak
        x_0 = box[1] / 2
        y_0 = box[2] / 2
        z_0 = box[0] / 2 # bead center should be the box center
        x_sig = 3
        y_sig = x_sig
        z_sig = 5
        p0 = [bg, A_coeff, z_0, x_0, y_0, z_sig, x_sig, y_sig]
        # params = []
        popt, pcov = curve_fit(
            matut.gaus_3D_for_fit_2,
            xdata,
            substack.ravel(), 
            p0)
        fit = matut.gaus_3D_for_fit_2(xdata, *popt)
        fit = np.reshape(fit, (box[0], box[1], box[2]))
        z_width.append(np.abs(popt[5]))
        x_width.append(np.abs(popt[6]))
        y_width.append(np.abs(popt[7]))

        mean_b_z += np.abs(popt[5]) / max_beads
        mean_b_x += np.abs(popt[6]) / max_beads
        mean_b_y += np.abs(popt[7]) / max_beads
        stack_1[
            int(z_chosen - box[0] / 2):int(z_chosen + box[0] / 2),
            int(x_max - box[1] / 2):int(x_max + box[1] / 2),
            int(y_max - box[2] / 2):int(y_max + box[2] / 2),
            ] = 0
        beads += 1
    else:
        stack_1[
            int(z_chosen - box[0] / 2):int(z_chosen + box[0] / 2),
            int(x_max - box[1] / 2):int(x_max + box[1] / 2),
            int(y_max - box[2] / 2):int(y_max + box[2] / 2),
            ] = 0
        
    print(beads, end=', ', flush=True)